In [1]:
import os 
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeout
import time

In [2]:
import asyncio
loop = asyncio.get_event_loop()

In [3]:
seasons = list(range(2018,2024))

In [4]:
seasons

[2018, 2019, 2020, 2021, 2022, 2023]

In [5]:
!pip install pytest-playwright

In [6]:
%pip install playwright-python

ERROR: Could not find a version that satisfies the requirement playwright-python (from versions: none)
ERROR: No matching distribution found for playwright-python
Note: you may need to restart the kernel to use updated packages.


In [7]:
%pip install beautifulsoup4


Note: you may need to restart the kernel to use updated packages.


In [8]:
%pip install playwright

Note: you may need to restart the kernel to use updated packages.


In [9]:
import os
os.environ['TMPDIR'] = '/Users/mohamedabuomar/NBA-Historical-Predictions/playwrightTemp'

In [10]:
!playwright install

In [11]:
DATA_DIR = "data"
STANDINGS_DIR = os.path.join(DATA_DIR, "Standings")
SCORES_DIR = os.path.join(DATA_DIR, "Scores")

In [12]:
STANDINGS_DIR

'data/Standings'

In [13]:
async def get_html(url, selector, sleep=20, retries=10):
    html = None
    for i in range(1, retries+1):
        time.sleep(sleep * i)
        try:
            os.environ['PLAYWRIGHT_BROWSERS_PATH'] = '/Users/mohamedabuomar/NBA-Historical-Predictions/playwrightTemp'
            async with async_playwright() as p:
                browser = await p.firefox.launch()
                page = await browser.new_page()
                await page.goto(url)
                print(await page.title())
                html = await page.inner_html(selector)
        except PlaywrightTimeout:
            print(f"Timeout error on {url}")
            continue
        else:
            break
    return html

In [14]:
season = 2016
url = f"https://www.basketball-reference.com/leagues/NBA_{season}_games.html"


In [15]:
url

'https://www.basketball-reference.com/leagues/NBA_2016_games.html'

In [16]:
html = await get_html(url, "#content .filter")


2015-16 NBA Schedule | Basketball-Reference.com


In [17]:
html

'\n\n<div class=" current">\n\t<a href="/leagues/NBA_2016_games-october.html">October</a>\n</div><div class="">\n\t<a href="/leagues/NBA_2016_games-november.html">November</a>\n</div><div class="">\n\t<a href="/leagues/NBA_2016_games-december.html">December</a>\n</div><div class="">\n\t<a href="/leagues/NBA_2016_games-january.html">January</a>\n</div><div class="">\n\t<a href="/leagues/NBA_2016_games-february.html">February</a>\n</div><div class="">\n\t<a href="/leagues/NBA_2016_games-march.html">March</a>\n</div><div class="">\n\t<a href="/leagues/NBA_2016_games-april.html">April</a>\n</div><div class="">\n\t<a href="/leagues/NBA_2016_games-may.html">May</a>\n</div><div class="">\n\t<a href="/leagues/NBA_2016_games-june.html">June</a>\n</div>'

In [18]:
 soup = BeautifulSoup(html)
links = soup.find_all("a")
href = [l['href'] for l in links]

In [19]:
href

['/leagues/NBA_2016_games-october.html',
 '/leagues/NBA_2016_games-november.html',
 '/leagues/NBA_2016_games-december.html',
 '/leagues/NBA_2016_games-january.html',
 '/leagues/NBA_2016_games-february.html',
 '/leagues/NBA_2016_games-march.html',
 '/leagues/NBA_2016_games-april.html',
 '/leagues/NBA_2016_games-may.html',
 '/leagues/NBA_2016_games-june.html']

In [20]:
async def scrape_season(season):
    url = f"https://www.basketball-reference.com/leagues/NBA_{season}_games.html"
    html = await get_html(url, "#content .filter")
    
    soup = BeautifulSoup(html)
    links = soup.find_all("a")
    standings_pages = [f"https://www.basketball-reference.com{l['href']}" for l in links]
    
    for url in standings_pages:
        save_path = os.path.join(STANDINGS_DIR, url.split("/")[-1])
        if os.path.exists(save_path):
            continue
        
        html = await get_html(url, "#all_schedule")
        with open(save_path, "w+") as f:
            f.write(html)

In [21]:
for season in seasons:
    await scrape_season(season)

2017-18 NBA Schedule | Basketball-Reference.com
2018-19 NBA Schedule | Basketball-Reference.com
2019-20 NBA Schedule | Basketball-Reference.com
2020-21 NBA Schedule | Basketball-Reference.com
2021-22 NBA Schedule | Basketball-Reference.com
2022-23 NBA Schedule | Basketball-Reference.com


In [22]:
check_schedule = os.listdir(STANDINGS_DIR)

In [23]:
check_schedule

['NBA_2022_games-october.html',
 'NBA_2021_games-june.html',
 'NBA_2020_games-march.html',
 'NBA_2020_games-september.html',
 'NBA_2020_games-january.html',
 'NBA_2020_games-august.html',
 'NBA_2023_games-may.html',
 'NBA_2019_games-april.html',
 'NBA_2023_games-april.html',
 'NBA_2022_games-may.html',
 'NBA_2019_games-february.html',
 'NBA_2018_games-february.html',
 'NBA_2023_games-january.html',
 'NBA_2021_games-march.html',
 'NBA_2018_games-january.html',
 'NBA_2018_games-april.html',
 'NBA_2020_games-december.html',
 'NBA_2019_games-october.html',
 'NBA_2020_games-november.html',
 'NBA_2021_games-may.html',
 'NBA_2021_games-december.html',
 'NBA_2022_games-april.html',
 'NBA_2020_games-october-2019.html',
 'NBA_2022_games-december.html',
 'NBA_2023_games-november.html',
 'NBA_2023_games-december.html',
 'NBA_2022_games-november.html',
 'NBA_2018_games-october.html',
 'NBA_2021_games-february.html',
 'NBA_2020_games-february.html',
 'NBA_2022_games-march.html',
 'NBA_2023_games-feb

In [27]:
async def scrape_game(check_schedule):
    with open(check_schedule, 'r') as f:
        html = f.read()

    soup = BeautifulSoup(html)
    links = soup.find_all("a")
    hrefs = [l.get('href') for l in links]
    box_scores = [f"https://www.basketball-reference.com{l}" for l in hrefs if l and "boxscore" in l and '.html' in l]

    for url in box_scores:
        save_path = os.path.join(SCORES_DIR, url.split("/")[-1])
        if os.path.exists(save_path):
            continue

        html = await get_html(url, "#content")
        if not html:
            continue
        with open(save_path, "w+") as f:
            f.write(html)

In [28]:
box_scores

NameError: name 'box_scores' is not defined

In [30]:
check_schedule = [s for s in check_schedule if ".html" in s]

for f in check_schedule:
    filepath = os.path.join(STANDINGS_DIR, f)

    await scrape_game(filepath)

Nets vs Bucks, October 19, 2021 | Basketball-Reference.com
Warriors vs Lakers, October 19, 2021 | Basketball-Reference.com
Pacers vs Hornets, October 20, 2021 | Basketball-Reference.com
Bulls vs Pistons, October 20, 2021 | Basketball-Reference.com
Celtics vs Knicks, October 20, 2021 | Basketball-Reference.com
Wizards vs Raptors, October 20, 2021 | Basketball-Reference.com
Cavaliers vs Grizzlies, October 20, 2021 | Basketball-Reference.com
Rockets vs Timberwolves, October 20, 2021 | Basketball-Reference.com
76ers vs Pelicans, October 20, 2021 | Basketball-Reference.com
Magic vs Spurs, October 20, 2021 | Basketball-Reference.com
Thunder vs Jazz, October 20, 2021 | Basketball-Reference.com
Kings vs Trail Blazers, October 20, 2021 | Basketball-Reference.com
Nuggets vs Suns, October 20, 2021 | Basketball-Reference.com
Mavericks vs Hawks, October 21, 2021 | Basketball-Reference.com
Bucks vs Heat, October 21, 2021 | Basketball-Reference.com
Clippers vs Warriors, October 21, 2021 | Basketball-

Error: NS_ERROR_UNKNOWN_HOST
=========================== logs ===========================
navigating to "https://www.basketball-reference.com/boxscores/201912040ATL.html", waiting until "load"
============================================================